# FPUS23 — DINO‑DETR SOTA (Despeckle + Optimized)

- Despeckle ultrasound images (if not already).
- Fine‑tune DINO‑DETR (HF) using COCO annotations + despeckled images.
- Save per epoch and auto‑resume.


In [ ]:
%pip -q install -U transformers datasets evaluate accelerate pycocotools huggingface_hub opencv-python-headless
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import os
REPO_URL = os.environ.get('FPUS23_REPO_URL', 'https://github.com/Srinivas-Raghav-VC/MultiFetalOrgan-Detection.git')
REPO_DIR = '/content/fpus23'
if not os.path.isdir(REPO_DIR):
    !git clone $REPO_URL $REPO_DIR
else:
    !git -C $REPO_DIR fetch && git -C $REPO_DIR pull --ff-only
%cd /content/fpus23
os.makedirs('/content/drive/MyDrive/FPUS23_runs', exist_ok=True)


In [ ]:
# Despeckle (idempotent)
from pathlib import Path
ORIG_YOLO_YAML = Path('/content/fpus23_project/dataset/fpus23_yolo/data.yaml')
IN_IMAGES = Path('/content/fpus23_project/dataset/fpus23_yolo/images')
DESP_IMG_ROOT = Path('/content/fpus23_project/dataset/fpus23_yolo_despeckled/images')
DESP_YAML = DESP_IMG_ROOT.parent / 'data.yaml'
if not DESP_YAML.exists():
    assert ORIG_YOLO_YAML.exists(), 'Original YOLO data.yaml missing; run prepare first.'
    !python '/content/fpus23/scripts/preprocess_fpus23_despeckle.py' \
      --input-dir $IN_IMAGES \
      --output-dir $DESP_IMG_ROOT \
      --kernel-size 5 \
      --splits train val test \
      --data-yaml $ORIG_YOLO_YAML
print('Using despeckled images at:', DESP_IMG_ROOT)


In [ ]:
# Train DINO-DETR with COCO JSONs + despeckled images
ANN_DIR = '/content/fpus23_project/dataset/fpus23_coco'
TRAIN_JSON = f'{ANN_DIR}/train.json'
VAL_JSON = f'{ANN_DIR}/val.json'
TRAIN_IMG = str(DESP_IMG_ROOT / 'train')
VAL_IMG = str(DESP_IMG_ROOT / 'val')
OUT_DIR = '/content/drive/MyDrive/FPUS23_runs/dinodetr_sota_despeckled'
!python '/content/fpus23/scripts/train_dinodetr_fpus23.py' \
  --train-json $TRAIN_JSON \
  --val-json $VAL_JSON \
  --train-images $TRAIN_IMG \
  --val-images $VAL_IMG \
  --epochs 30 \
  --batch 8 \
  --lr 5e-5
